In [1]:
import os
import json
import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [5]:
!unzip -qn /content/drive/MyDrive/TEST_DATA_SET-001.zip -d /content/test/
!unzip -qn /content/drive/MyDrive/TEST_DATA_SET-002.zip -d /content/test/

In [8]:
TEST_JSON_PATH = '/content/test/TEST_DATA_SET/label (라벨링)/'

TEST_IMG_PATH = '/content/test/TEST_DATA_SET/'

In [9]:
with open (TEST_JSON_PATH + "test_anger.json", "r", encoding='euc-kr') as f:
    anger_data = json.load(f)
with open (TEST_JSON_PATH + "test_happy.json", "r", encoding='euc-kr') as f:
    happy_data = json.load(f)
with open (TEST_JSON_PATH + "test_panic.json", "r", encoding='euc-kr') as f:
    panic_data = json.load(f)
with open (TEST_JSON_PATH + "test_sadness.json", "r", encoding='euc-kr') as f:
    sadness_data = json.load(f)

In [10]:
# JSON 데이터를 DataFrame으로 변환
anger_df = pd.DataFrame(anger_data)
happy_df = pd.DataFrame(happy_data)
panic_df = pd.DataFrame(panic_data)
sadness_df = pd.DataFrame(sadness_data)

In [20]:
# 각 DataFrame을 리스트에 저장
dfs = [anger_df, happy_df, panic_df, sadness_df]

# DataFrame 합치기
test_df = pd.concat(dfs, ignore_index=True)
test_df = test_df.sort_values(by='filename')
test_df.head()

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
1086,00riaf6eb09fb4924b3a7f4b37d3af4870b733b5bfaa89...,여,20,일반인,슬픔,실외 자연환경,"{'boxes': {'maxX': 4303.291545743236, 'maxY': ...","{'boxes': {'maxX': 4376.479999999999, 'maxY': ...","{'boxes': {'maxX': 4376.48, 'maxY': 2908.31739..."
695,01h6ce85f03f4589312446c62372f5cb24ea3083e41d2b...,남,30,전문인,당황,상업시설/점포/시장,"{'boxes': {'maxX': 2396.3595570216853, 'maxY':...","{'boxes': {'maxX': 2419.934266408367, 'maxY': ...","{'boxes': {'maxX': 2400.1004080922257, 'maxY':..."
0,04b68c38839513fad12d5d85383622e2eaffbf77461a6a...,남,20,전문인,분노,상업시설/점포/시장,"{'boxes': {'maxX': 2032.2349, 'maxY': 1836.338...","{'boxes': {'maxX': 2064.234777774114, 'maxY': ...","{'boxes': {'maxX': 2066.410411001555, 'maxY': ..."
1,066ode77d7bb7d3c007199fdccc0dededadd00c9c984b3...,남,30,전문인,분노,도심 환경,"{'boxes': {'maxX': 1563.9583, 'maxY': 1043.875...","{'boxes': {'maxX': 1543.2924683994665, 'maxY':...","{'boxes': {'maxX': 1564.1165697987633, 'maxY':..."
994,06sl9a51898007641b2ac9b8a18645ea25950c4acda182...,여,30,전문인,슬픔,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 2237.757110216773, 'maxY': ...","{'boxes': {'maxX': 2234.1227693435812, 'maxY':...","{'boxes': {'maxX': 2226.8270454347967, 'maxY':..."


In [40]:
sadness_df = sadness_df.sort_values(by='filename')
sadness_df.reset_index(drop=True, inplace=True)
sadness_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,00riaf6eb09fb4924b3a7f4b37d3af4870b733b5bfaa89...,여,20,일반인,슬픔,실외 자연환경,"{'boxes': {'maxX': 4303.291545743236, 'maxY': ...","{'boxes': {'maxX': 4376.479999999999, 'maxY': ...","{'boxes': {'maxX': 4376.48, 'maxY': 2908.31739..."
1,06sl9a51898007641b2ac9b8a18645ea25950c4acda182...,여,30,전문인,슬픔,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 2237.757110216773, 'maxY': ...","{'boxes': {'maxX': 2234.1227693435812, 'maxY':...","{'boxes': {'maxX': 2226.8270454347967, 'maxY':..."
2,0cyn350e7b9f3be037d2b7747cc267375f3cf3f1f36e53...,여,20,일반인,슬픔,숙박 및 거주공간,"{'boxes': {'maxX': 2800.3032000000003, 'maxY':...","{'boxes': {'maxX': 2789.653253694737, 'maxY': ...","{'boxes': {'maxX': 2800.3032, 'maxY': 1692.400..."
3,0hjp0ed05edcce6560cb5aca50ed588a9030a3deae64ad...,남,10,일반인,슬픔,숙박 및 거주공간,"{'boxes': {'maxX': 2073.403, 'maxY': 1506.9808...","{'boxes': {'maxX': 2095.276417721519, 'maxY': ...","{'boxes': {'maxX': 2073.403, 'maxY': 1506.9807..."
4,0kldc0396782e2f25760f565f88712efb96270c07cb600...,여,40,일반인,슬픔,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 1831.830598, 'maxY': 1633.1...","{'boxes': {'maxX': 1829.516598, 'maxY': 1637.5...","{'boxes': {'maxX': 1826.1445979999999, 'maxY':..."
...,...,...,...,...,...,...,...,...,...
279,z0lw1199f1478f5b2fbf6e5f6805266199e5e09e62ddba...,여,30,전문인,슬픔,실외 자연환경,"{'boxes': {'maxX': 2355.1811999999995, 'maxY':...","{'boxes': {'maxX': 2355.0683762118747, 'maxY':...","{'boxes': {'maxX': 2355.1812, 'maxY': 1365.813..."
280,z0z3700a2801a886162a6fda72f9e0e7ad98d517e62777...,여,20,전문인,슬픔,실외 자연환경,"{'boxes': {'maxX': 909.6957400000001, 'maxY': ...","{'boxes': {'maxX': 928.9408054245686, 'maxY': ...","{'boxes': {'maxX': 909.69574, 'maxY': 763.948,..."
281,zg3xd0ee6311768828989324f5016377222a6377c04762...,남,30,전문인,슬픔,행사/사무공간,"{'boxes': {'maxX': 1591.254, 'maxY': 1379.0898...","{'boxes': {'maxX': 1604.1500528612346, 'maxY':...","{'boxes': {'maxX': 1609.213505599148, 'maxY': ..."
282,zsbr1b313b1fd7bfab44af8337422c897efe971575e207...,여,30,일반인,슬픔,도심 환경,"{'boxes': {'maxX': 2370.716238554217, 'maxY': ...","{'boxes': {'maxX': 2338.6415819310105, 'maxY':...","{'boxes': {'maxX': 2338.611488696329, 'maxY': ..."


In [42]:
# 디렉토리 내 이미지 파일 경로
directory_path = '/content/test/TEST_DATA_SET/4. sadness'
image_files = sorted(os.listdir(directory_path))


def save_region_as_image(original_image, save_directory, file_name):
    try:
        image = original_image
        # 좌표 가져오기
        maxX = sadness_df['annot_A'][i]['boxes']['maxX']
        maxY = sadness_df['annot_A'][i]['boxes']['maxY']
        minX = sadness_df['annot_A'][i]['boxes']['minX']
        minY = sadness_df['annot_A'][i]['boxes']['minY']

        # 이미지 자르기
        image = image[int(minY):int(maxY), int(minX):int(maxX)]

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, image)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")


for i, file_name in enumerate(image_files):
    if file_name in image_files:
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/sadness_crop/'
        save_file_name = f'sadness{i+1}.jpg'
        save_region_as_image(original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")